## The goals for this Advanced Lane Finding Project project are:

### -- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
### -- Apply a distortion correction to raw images.
### -- Use color transforms, gradients, etc., to create a thresholded binary image.
### -- Apply a perspective transform to rectify binary image ("birds-eye view").
### -- Detect lane pixels and fit to find the lane boundary.
### -- Determine the curvature of the lane and vehicle position with respect to center.
### -- Warp the detected lane boundaries back onto the original image.
### -- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

### Python Code for the Advanced Lane Finding project:

### Import the necessary packages

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import importlib

#from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import h5py
import os

In [4]:
# Take an image, object points, image points, and perform the camera calibration. Undistort the image after camera calibration
        
#load image for reference
#image = cv2.imread('./camera_cal/calibration1.jpg')
#img_size = (image.shape[1],image.shape[0])

# Perform camera calibration with the given object and image points
#ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

# Save the camera calibration results for later use
filename = "camera_intrinsics.h5"
main_K, main_distortion = None, None

#Extracts the intrinsic K matrix and distortion vector from the h5 file
with h5py.File(filename, "r") as data:

    main_K = np.array(data["main"]["K"])
    main_distortion = np.array(data["main"]["distortion"])
dist_pickle = {}
k = main_K
dst = main_distortion
pickle.dump(dist_pickle, open("./camera_cal/calibration_pickle.p", "wb"))


In [6]:
# Read and make a list of test images
images = glob.glob('final_5/*.png')
images.sort()
print(images)
undist = {}
for image_name in images:
    img = cv2.imread(image_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    undist[image_name] = img
    
    #plt.figure(figsize = (18,12))

    grid = gridspec.GridSpec(5,1)

    #plt.subplot(grid[0])
    #plt.imshow(undist[image_name])

###plt.show()

['final_5/proc_0141.png', 'final_5/proc_0802.png', 'final_5/proc_0825.png', 'final_5/proc_0848.png', 'final_5/proc_0902.png']


### Although it may not be visible at a first glance, a closer look at the sides of the undistorted image above shows that the radial distortion has been removed. An example where this is obvious: the white car on the right is slightly cropped along with the trees

### Some useful functions are defined for experimenting with different color thresholds and gradients

In [203]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.

def abs_sobel_thresh(img, orient='x', thresh_min=25, thresh_max=255):
    # Convert to grayscale
    # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(l_channel, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(l_channel, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def color_threshold(image, sthresh=(0,255), vthresh=(0,255), hthresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > sthresh[0]) & (s_channel <= sthresh[1])] = 1

    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel > vthresh[0]) & (v_channel <= vthresh[1])] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h_channel = hsv[:,:,0]
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel > hthresh[0]) & (h_channel <= hthresh[1])] = 1

    output = np.zeros_like(s_channel)
    output[(s_binary == 1) & (v_binary == 1) & (h_binary == 1)] = 1

    # Return the combined s_channel & v_channel binary image
    return output

def s_channel_threshold(image, sthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:, :, 2]  # use S channel

    # create a copy and apply the threshold
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel >= sthresh[0]) & (s_channel <= sthresh[1])] = 1
    return binary_output

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level*(height/4) + height)):int(img_ref.shape[0]-level*(height/4)), max(0,int(center-width)):min(int(center+width),img_ref.shape[1])] = 1
    return output

In [204]:
gradx = {}
for image_name in undist.keys():
    #Apply Sobel operator in X-direction to experiment with gradient thresholds
    gradx[image_name] = abs_sobel_thresh(undist[image_name], orient='x', thresh_min=7, thresh_max=100)

    #Visualize the results before/after absolute sobel operator is applied on a test image in x direction to find the
    #vertical lines, since the lane lines are close to being vertical
    #plt.figure(figsize = (18,12))
    grid = gridspec.GridSpec(1,2)

    # set the spacing between axes.
    grid.update(wspace=0.1, hspace=0.1)  

    #plt.subplot(grid[0])
    #plt.imshow(undist[image_name], cmap="gray")
    #plt.title('Undistorted Image')

    #plt.subplot(grid[1])
    #plt.imshow(gradx[image_name], cmap="gray")
    #plt.title('Absolute sobel threshold in X direction')

#    ##plt.show()

In [205]:
c_binary = {}
for image_name in undist.keys():
    #Experiment with HLS & HSV color spaces along with thresholds
    c_binary[image_name] = color_threshold(undist[image_name], hthresh=(0,70))

    #Visualize the results before/after HLS/HSV  threshold is applied
    #plt.figure(figsize = (18,12))
    grid = gridspec.GridSpec(1,2)

    # set the spacing between axes.
    grid.update(wspace=0.1, hspace=0.1)  

    #plt.subplot(grid[0])
    #plt.imshow(undist[image_name], cmap="gray")
    #plt.title('Undistorted Image')

    #plt.subplot(grid[1])
    #plt.imshow(c_binary[image_name], cmap="gray")
    #plt.title('After applying color threshold')

##plt.show()

### After experimenting with several thresholds and color spaces, I chose to use the combined binary thresholded image from the Sobel threshold in the x & y directions along with the color thresholds in the H & V channels, to get clear lane lines in all the test images. This forms the image processing pipeline for generating a thresholded binary image

In [206]:
sobel_color = {}
for image_name in undist.keys():
    #Combine the binary images using the Sobel thresholds in X/Y directions along with the color threshold to form the final image pipeline
    sobel_color[image_name] = np.zeros_like(undist[image_name][:,:,0])
    sobel_color[image_name][((gradx[image_name] == 1) | (c_binary[image_name] == 1))] = 255

    #Visualize the results before/after combining the images from the pipeline
    #plt.figure(figsize = (18,12))
    grid = gridspec.GridSpec(1,2)

    # set the spacing between axes.
    grid.update(wspace=0.1, hspace=0.1)  

    #plt.subplot(grid[0])
    #plt.imshow(undist[image_name], cmap="gray")
    #plt.title('Undistorted Image')

    #plt.subplot(grid[1])
    #plt.imshow(sobel_color[image_name], cmap="gray")
    #plt.title('After image processing pipeline')

##plt.show()

#processed = sobel_color

In [207]:
processed = {}
best_horizon = {}
th_low = -np.pi / 8
th_high = np.pi / 8
th_step = np.pi / 32
b_low = 50
b_high = 88
b_step = 4
"""
#plt.imshow(sobel_color[image_name], cmap="gray")
for th in np.arange(th_low, th_high, th_step):
        for b in np.arange(b_low, b_high, b_step):
            m = np.tan(th)
            print('(', m, ', ', b, ')')
            #plt.plot((0, 50), (b, b + 50 * m),  marker='o')
##plt.plot((25, 100), (0, 50), marker='o')
##plt.show()
"""
for image_name in sobel_color.keys():
    best_th = -1
    best_b  = -1
    best_sum = -1
    img = sobel_color[image_name]
    img[:,int( 0.5 * img.shape[1]):] = 0 # Crop out a little less than the right half of the image
    for th in np.arange(th_low, th_high, th_step):
        for b in np.arange(b_low, b_high, b_step):
            csum = 0
            m = np.tan(th)
            for x in range(img.shape[0]):
                for y in range(int(img.shape[1] / 2)):
                    #print('(', x, y, ') ', img[x][y])
                    if (x > m * y + b and img[x][y] == 255) or (x < m * y + b and img[x][y] == 0):
                        csum -= 1
                    else:
                        csum += 1
            csum = abs(csum)
            if csum > best_sum:
                best_sum = csum
                best_th = th
                best_b = b
    best_img = np.zeros_like(img)
    best_m = np.tan(best_th)
    best_horizon[image_name] = (best_m, best_b)            

    for x in range(img.shape[0]):
        for y in range(int(img.shape[1] / 2)):
            #print('(', x, y, ') ', img[x][y])
            if x < best_m * y + best_b:
                best_img[x][y] = 0
            else:
                best_img[x][y] = img[x][y]
    processed[image_name] = best_img

    #Visualize the results before/after combining the images from the pipeline
    #plt.figure(figsize = (18,12))
    grid = gridspec.GridSpec(1,2)

    # set the spacing between axes.
    grid.update(wspace=0.1, hspace=0.1)  

    #plt.subplot(grid[0])
    print('best_th ', best_th, '; best_b ', best_b)
    #plt.imshow(sobel_color[image_name], cmap="gray")
    #plt.title('Sobel / Color Image')

    #plt.subplot(grid[1])
    #plt.imshow(processed[image_name], cmap="gray")
    #plt.title('After image processing pipeline')

##plt.show()


best_th  -0.09817477042468103 ; best_b  82
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  74
best_th  -0.09817477042468103 ; best_b  74
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  74
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  74
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.19634954084936207 ; best_b  86
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0.09817477042468103 ; best_b  78
best_th  -0

In [239]:
for k in processed.keys():
    cv2.imwrite('processed_images/processed_' + k[-8:], processed[k])
for k in processed:
    print('processed_images/processed_' + k[-8:])
    print(processed[k])

processed_images/processed_0004.png
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0 255 255 ...   0   0   0]
 [  0 255 255 ...   0   0   0]
 [  0 255 255 ...   0   0   0]]
processed_images/processed_0019.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
processed_images/processed_0050.png
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0 255 ...   0   0   0]
 [  0 255 255 ...   0   0   0]
 [  0 255 255 ...   0   0   0]]
processed_images/processed_0052.png
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 [  0 255 255 ...   0   0   0]]
processed_images/processed_0067.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
proc

 [0 0 0 ... 0 0 0]]
processed_images/processed_1023.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
processed_images/processed_1027.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
processed_images/processed_1031.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
processed_images/processed_1038.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Read the test images, pass them through the pipeline, warp the perspective by selecting a region of interest within the image, and create a bird's eye view. Visualize the results and ensure that the selected region of interest is appropriate by confirming that the lane lanes are indeed parallel to each other after warping the image

### Apply convolution which will maximize the number of "hot" pixels in each window. This convolution is the summation of the product of two separate signals: the window template and the vertical slice of the pixel image. The window template is slid across the image from left to right and any overlapping values are summed together, creating the convolved signal. The peak of the convolved signal is where the highest overlap of pixels occured and is the position for the lane marker.

In [208]:
gidx = 0

def window_sliding(warped):
    window_width = 7
    window_height = 8
    
    #set up the overall class to do the lane line tracking
    curve_centers = tracker.tracker(hit_fraction = 0.56, Mymargin = 20, Mywindow_width=window_width, Mywindow_height=window_height, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor=15)
    
    window_centroids = curve_centers.find_window_centroids(warped, best_horizon[image_name])
    
    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
        
    # points used to find the right & left lanes
    leftp = []

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        # Add center value found in frame to the list of lane points per left, right
        leftp.append(window_centroids[level])

        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
    # Draw the results
    template = np.array(l_points,np.uint8) #template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the original road image with window results

    #print(image_name)

    #Visualize the results 
    #plt.figure(figsize = (30,20))
    grid = gridspec.GridSpec(5,1)
    # set the spacing between axes.
    grid.update(wspace=0.05, hspace=0.05)  

    #plt.subplot(grid[gidx])
    #plt.imshow(result, cmap='gray')
    #plt.title('Window fitting results')
    return leftp

In [217]:
for x in leftp_d:
    print(x, leftp_d[x], '\n')

../proc_main/proc_0004.png [(4.5, 100), (22.5, 98)] 

../proc_main/proc_0019.png [(145.5, 100), (147.5, 98), (147.5, 96), (149.5, 94), (149.5, 92), (149.5, 90), (149.5, 88)] 

../proc_main/proc_0050.png [(123.5, 100), (123.5, 98), (125.5, 96), (137.5, 94), (135.5, 92), (135.5, 90), (135.5, 88), (135.5, 86), (136.5, 84), (136.5, 82), (136.5, 80), (137.5, 78), (137.5, 76), (139.5, 74), (138.5, 72), (138.5, 70), (138.5, 68)] 

../proc_main/proc_0052.png [(124.5, 100), (125.5, 98), (126.5, 96), (129.5, 94), (141.5, 92), (132.5, 90), (132.5, 88), (136.5, 86), (136.5, 84), (136.5, 82), (136.5, 80), (137.5, 78), (138.5, 76), (139.5, 74), (139.5, 72), (138.5, 70)] 

../proc_main/proc_0067.png [(124.5, 100), (124.5, 98), (133.5, 96), (133.5, 94), (133.5, 92), (132.5, 90), (135.5, 88), (136.5, 86), (136.5, 84), (137.5, 82), (137.5, 80), (138.5, 78), (138.5, 76), (138.5, 74), (137.5, 72)] 

../proc_main/proc_0070.png [(123.5, 100), (124.5, 98), (134.5, 96), (136.5, 94), (133.5, 92), (133.5, 90), 

In [2]:
leftp_d = {}
import tracker

for image_name in processed.keys():  
    leftp_d[image_name] = window_sliding(processed[image_name])

##plt.show()

NameError: name 'processed' is not defined

### Fit a polynomial to the identified lane lines on the left and the right. Visualize the results by overlapping the lane lines on to the original undistorted image

In [1]:
gidx=0
lane_polynomial_d = {}
final_images = {}
for image_name in leftp_d.keys():
    img = undist[image_name]
    #fit the lane boundaries to the left, right center positions found
    yvals = range(int(leftp_d[image_name][-1][1]),warped.shape[0]) # leftp_d[image_name][-1][1] represents the highest window centroid
    
    res_yvals = [x[1] for x in leftp_d[image_name]]
    leftx = [x[0] for x in leftp_d[image_name]]
    
    
    #print('len(res_rvals)', res_yvals, '\nlen leftx', leftx)
    #print('\n', warped.shape)
    left_fit = np.polyfit(res_yvals, leftx, 3) # DEGREE OF POLYNOMIAL = 2
    left_fitx = left_fit[0]*yvals*yvals*yvals + left_fit[1]*yvals*yvals + left_fit[2]*yvals + left_fit[3] # NEED TO CHANGE BASED ON POLYNOMIAL DEGREE
    left_fitx = np.array(left_fitx,np.int32)
    
    lane_polynomial_d[image_name] = {}
    lane_polynomial_d[image_name]['coefficients'] = left_fit
    lane_polynomial_d[image_name]['y'] = yvals
    lane_polynomial_d[image_name]['x'] = left_fitx    
    
    """
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    right_fitx = np.array(right_fitx,np.int32)
    """
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/8, left_fitx[::-1]+window_width/8),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    #right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)

    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    cv2.fillPoly(road,[left_lane],color=[0,0,255])
    #cv2.fillPoly(road,[right_lane],color=[0,0,255])
    cv2.fillPoly(road_bkg,[left_lane],color=[0,0,255])
    #cv2.fillPoly(road_bkg,[right_lane],color=[255,255,255])

    road_warped = cv2.warpPerspective(road,Minv,img_size,flags=cv2.INTER_LINEAR)
    road_warped_bkg= cv2.warpPerspective(road_bkg,Minv,img_size,flags=cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(img,1.0,road, -1.0, 0.0)
    result = cv2.addWeighted(base,1.0,road, 1.0, 0.0)

    #Visualize the results of identified lane lines and overlapping them on to the original undistorted image
    #plt.figure(figsize = (30,20))
    grid = gridspec.GridSpec(8,2)
    # set the spacing between axes.
    grid.update(wspace=0.05, hspace=0.05)  

    #img_#plt = #plt.subplot(grid[0])
    #plt.subplot(grid[gidx])
    #plt.imshow(road, cmap="gray")
    #plt.scatter(leftx, res_yvals)
    #plt.title('Identified lane lines')

    #img_#plt = #plt.subplot(grid[1])
    #plt.subplot(grid[gidx+1])
    #plt.imshow(result, cmap="gray")
    #plt.title('Lane lines overlapped on original image')
    final_images[image_name] = result
    
##plt.show()

NameError: name 'leftp_d' is not defined

In [235]:
for k in final_images.keys():
    cv2.imwrite('final_images/final_' + k[-8:], final_images[k])
for k in final_images:
    print('final_images/final_' + k[-8:])
    print(final_images[k])

final_images/final_0004.png
[[[107 102 100]
  [ 93  87  85]
  [ 82  78  77]
  ...
  [ 98  94  91]
  [ 87  84  82]
  [ 83  80  79]]

 [[120 118 116]
  [116 114 111]
  [101  98  96]
  ...
  [ 77  73  70]
  [ 71  68  67]
  [ 67  64  63]]

 [[ 80  77  75]
  [103 101  99]
  [123 122 120]
  ...
  [ 63  59  58]
  [ 58  55  54]
  [ 53  51  50]]

 ...

 [[138 149 255]
  [138 148 255]
  [131 138 255]
  ...
  [ 95  97  96]
  [ 94  96  95]
  [ 92  95  94]]

 [[115 120 118]
  [109 112 112]
  [108 111 109]
  ...
  [ 94  96  94]
  [ 92  94  92]
  [ 89  92  90]]

 [[135 142 142]
  [136 142 142]
  [135 140 139]
  ...
  [ 95  97  96]
  [ 93  95  93]
  [ 88  92  88]]]
final_images/final_0019.png
[[[ 88  87  83]
  [ 69  66  64]
  [ 70  67  66]
  ...
  [132 131 127]
  [142 140 135]
  [ 59  56  54]]

 [[ 95  93  89]
  [ 78  75  74]
  [ 74  72  71]
  ...
  [124 123 119]
  [144 143 137]
  [ 63  61  59]]

 [[ 98  96  94]
  [ 82  80  78]
  [ 77  75  74]
  ...
  [122 121 118]
  [142 144 140]
  [ 67  67  66]]

 .

[[[ 52  51  49]
  [ 52  51  49]
  [ 72  71  68]
  ...
  [ 99 101  99]
  [ 91  91  90]
  [102 102 101]]

 [[ 52  50  48]
  [ 56  54  53]
  [ 73  71  70]
  ...
  [100 101  98]
  [ 93  93  91]
  [ 98  98  96]]

 [[ 41  40  39]
  [ 49  48  47]
  [ 69  68  66]
  ...
  [100 101  99]
  [ 94  93  91]
  [ 93  93  91]]

 ...

 [[103 106 105]
  [104 107 106]
  [103 106 105]
  ...
  [ 53  50  48]
  [ 56  53  52]
  [ 61  58  57]]

 [[105 108 107]
  [105 108 107]
  [104 107 106]
  ...
  [ 58  55  52]
  [ 58  55  54]
  [ 61  58  57]]

 [[106 109 108]
  [106 109 108]
  [106 109 108]
  ...
  [ 61  59  56]
  [ 59  56  55]
  [ 60  57  56]]]
final_images/final_0326.png
[[[ 53  53  51]
  [ 75  75  73]
  [ 84  83  80]
  ...
  [131 133 131]
  [125 127 124]
  [ 93  93  91]]

 [[ 52  53  50]
  [ 74  74  72]
  [ 85  84  82]
  ...
  [130 132 131]
  [129 130 128]
  [ 92  92  90]]

 [[ 48  49  46]
  [ 71  72  69]
  [ 84  84  82]
  ...
  [128 130 129]
  [131 131 130]
  [ 92  92  90]]

 ...

 [[103 106 105]
  [103 1

final_images/final_1038.png
[[[108 106 103]
  [116 114 111]
  [117 115 111]
  ...
  [110 108 106]
  [127 127 125]
  [127 126 125]]

 [[ 77  74  73]
  [ 97  95  92]
  [108 107 103]
  ...
  [118 117 114]
  [127 127 125]
  [139 138 136]]

 [[ 52  49  48]
  [ 55  53  51]
  [ 71  69  67]
  ...
  [114 113 110]
  [139 139 138]
  [143 146 144]]

 ...

 [[123 130 130]
  [124 131 131]
  [122 129 129]
  ...
  [ 69  67  66]
  [ 63  60  59]
  [ 61  58  57]]

 [[123 130 130]
  [123 130 130]
  [123 130 130]
  ...
  [ 63  60  59]
  [ 63  60  59]
  [ 62  59  58]]

 [[124 130 131]
  [124 130 131]
  [125 131 131]
  ...
  [ 62  59  58]
  [ 63  59  58]
  [ 61  57  56]]]


In [213]:
# Saving the polynomials found into pickle file
"""
for x in lane_polynomial_d:
    print('IMAGE ', x, '\n')
    for k in lane_polynomial_d[x]:
        print(k, lane_polynomial_d[x][k], '\n')
"""

"""
import pickle
with open("lane_polynomials.obj", "wb") as file:  
    pickle.dump(lane_polynomial_d, file)
    outfile.close()
"""

'\nimport pickle\nwith open("lane_polynomials.obj", "wb") as file:  \n    pickle.dump(lane_polynomial_d, file)\n    outfile.close()\n'

In [214]:
# Loading the lane polynomials pickle file
"""
with open("lane_polynomials.obj", "rb") as file:  
    ld = pickle.load(file)
    file.close()
print(ld)
"""

'\nwith open("lane_polynomials.obj", "rb") as file:  \n    ld = pickle.load(file)\n    file.close()\nprint(ld)\n'

### Calculate the radius of curvature using polynomial fit functions, and the position of the camera/car's center from the left or right lane. Display these results along with the fitted lane lines on top of the original image

In [215]:
gidx=0
for idx,fname in enumerate(images):
    #read in image
    img = cv2.imread(fname)
    #undistort the image
    img = cv2.undistort(img,mtx,dist,None,mtx)
    
    preprocessImage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh_min=12, thresh_max=255)
    grady = abs_sobel_thresh(img, orient='y', thresh_min=25, thresh_max=255)
    c_binary = color_threshold(img, sthresh=(100,255), vthresh=(50,255))
    preprocessImage[((gradx == 1) & (grady ==1) | (c_binary == 1))] = 255

    img_size = (img.shape[1],img.shape[0])
    
    bot_width = .76 # percentage of bottom trapezoidal height
    mid_width = .08 # percentage of mid trapezoidal height
    height_pct = .62 # percentage of trapezoidal height
    bottom_trim= .935 # percentage from top to bottom avoiding the hood of the car
    
    src = np.float32([[img.shape[1]*(0.5-mid_width/2), img.shape[0]*height_pct],[img.shape[1]*(0.5+mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(0.5+bot_width/2), img.shape[0]*bottom_trim],[img.shape[1]*(0.5-bot_width/2), img.shape[0]*bottom_trim]])
    offset = img_size[0]*0.25
    dst = np.float32([[offset,0],[img_size[0]-offset,0],[img_size[0]-offset,img_size[1]],[offset,img_size[1]]])   
    
    #perform the warp perspective transform
    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(preprocessImage, M, img_size, flags=cv2.INTER_LINEAR)
    
    window_width = 25
    window_height = 80
    
    #set up the overall class to do the lane line tracking
    curve_centers = tracker(Mywindow_width=window_width, Mywindow_height=window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor=15)
    
    window_centroids = curve_centers.find_window_centroids(warped)
    
    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
        
    # points used to find the right & left lanes
    rightx = []
    leftx = []

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        # Add center value found in frame to the list of lane points per left, right
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the original road image with window results
    
    #fit the lane boundaries to the left, right center positions found
    yvals = range(0,warped.shape[0])
    
    res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)
    
    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx,np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    right_fitx = np.array(right_fitx,np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2, left_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)

    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    cv2.fillPoly(road,[left_lane],color=[255,0,0])
    cv2.fillPoly(road,[right_lane],color=[0,0,255])
    cv2.fillPoly(road_bkg,[left_lane],color=[255,255,255])
    cv2.fillPoly(road_bkg,[right_lane],color=[255,255,255])

    road_warped = cv2.warpPerspective(road,Minv,img_size,flags=cv2.INTER_LINEAR)
    road_warped_bkg= cv2.warpPerspective(road_bkg,Minv,img_size,flags=cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(img,1.0,road_warped, -1.0, 0.0)
    result = cv2.addWeighted(base,1.0,road_warped, 1.0, 0.0)
    ym_per_pix = curve_centers.ym_per_pix # meters per pixel in y dimension
    xm_per_pix = curve_centers.xm_per_pix # meters per pixel in x dimension

    curve_fit_cr = np.polyfit(np.array(res_yvals,np.float32)*ym_per_pix,np.array(leftx,np.float32)*xm_per_pix,2)
    curverad = ((1 + (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])
    
    # Calculate the offset of the car on the road
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pos = 'right'

    # draw the text showing curvature, offset & speed
    cv2.putText(result, 'Radius of Curvature='+str(round(curverad,3))+'m ',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

    #plt.imshow(result, cmap='gray')
    #plt.title('Final image results')
    ##plt.show()
    
    write_name='./test_images/tracked'+str(idx)+'.jpg'
    cv2.imwrite(write_name, result)

NameError: name 'mtx' is not defined

### Process videos to display the fitted lane lines, radius of curvature, and position of the car from the center

In [ ]:
# Set up the process videos function
def process_image(img):
    #undistort the image
    img = cv2.undistort(img,mtx,dist,None,mtx)
    
    warptrap = np.copy(img)
    cv2.line(warptrap, (int(src[0][0]), int(src[0][1])), (int(src[1][0]), int(src[1][1])), [255,0,0], 10, cv2.LINE_AA)
    cv2.line(warptrap, (int(src[1][0]), int(src[1][1])), (int(src[2][0]), int(src[2][1])), [255,0,0], 10, cv2.LINE_AA)
    cv2.line(warptrap, (int(src[2][0]), int(src[2][1])), (int(src[3][0]), int(src[3][1])), [255,0,0], 10, cv2.LINE_AA)
    cv2.line(warptrap, (int(src[3][0]), int(src[3][1])), (int(src[0][0]), int(src[0][1])), [255,0,0], 10, cv2.LINE_AA)
    
    #pass image thru the pipeline
    #preprocessImage = np.zeros_like(img[:,:,0])
    #gradx = abs_sobel_thresh(img, orient='x',thresh_min=20, thresh_max=100)
    #s_binary = s_channel_threshold(img, sthresh=(170,255))
    #preprocessImage[((gradx == 1) | (s_binary == 1))] = 1
    preprocessImage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh_min=12, thresh_max=255)
    grady = abs_sobel_thresh(img, orient='y', thresh_min=25, thresh_max=255)
    c_binary = color_threshold(img, sthresh=(100,255), vthresh=(50,255))
    preprocessImage[((gradx == 1) & (grady ==1) | (c_binary == 1))] = 255

    binaryImage = np.copy(preprocessImage)
    binaryImage = np.array(cv2.merge((binaryImage,binaryImage,binaryImage)),np.uint8)
    cv2.putText(binaryImage, 'Binary Image',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    
    img_size = (img.shape[1],img.shape[0])
    
    bot_width = .76 # percentage of bottom trapezoidal height
    mid_width = .08 # percentage of mid trapezoidal height
    height_pct = .62 # percentage of trapezoidal height
    bottom_trim= .935 # percentage from top to bottom avoiding the hood of the car
    
    src = np.float32([[img.shape[1]*(0.5-mid_width/2), img.shape[0]*height_pct],[img.shape[1]*(0.5+mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(0.5+bot_width/2), img.shape[0]*bottom_trim],[img.shape[1]*(0.5-bot_width/2), img.shape[0]*bottom_trim]])
    offset = img_size[0]*0.25
    dst = np.float32([[offset,0],[img_size[0]-offset,0],[img_size[0]-offset,img_size[1]],[offset,img_size[1]]])   
    
    #perform the warp perspective transform
    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(preprocessImage, M, img_size, flags=cv2.INTER_LINEAR)
    
    window_width = 25
    window_height = 80
    
    #set up the overall class to do the lane line tracking
    curve_centers = tracker(Mywindow_width=window_width, Mywindow_height=window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor=15)
    
    window_centroids = curve_centers.find_window_centroids(warped)
    
    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
        
    # points used to find the right & left lanes
    rightx = []
    leftx = []

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        # Add center value found in frame to the list of lane points per left, right
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the original road image with window results

    windowfit = np.copy(result)
    cv2.putText(windowfit, 'Sliding window results',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    
    warpage1 = np.copy(warpage)
    cv2.putText(warpage1, 'Bird\'s-eye View',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.line(warpage1, (int(dst[0][0]), int(dst[0][1])), (int(dst[1][0]), int(dst[1][1])), [0,0,255], 10, cv2.LINE_AA)
    cv2.line(warpage1, (int(dst[1][0]), int(dst[1][1])), (int(dst[2][0]), int(dst[2][1])), [0,0,255], 10, cv2.LINE_AA)
    cv2.line(warpage1, (int(dst[2][0]), int(dst[2][1])), (int(dst[3][0]), int(dst[3][1])), [0,0,255], 10, cv2.LINE_AA)
    cv2.line(warpage1, (int(dst[3][0]), int(dst[3][1])), (int(dst[0][0]), int(dst[0][1])), [0,0,255], 10, cv2.LINE_AA)
    
    #fit the lane boundaries to the left, right center positions found
    yvals = range(0,warped.shape[0])
    
    res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)
    
    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx,np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    right_fitx = np.array(right_fitx,np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2, left_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    inner_lane = np.array(list(zip(np.concatenate((left_fitx+window_width/2, right_fitx[::-1]-window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)

    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    cv2.fillPoly(road,[left_lane],color=[255,0,0])
    cv2.fillPoly(road,[right_lane],color=[0,0,255])
    cv2.fillPoly(road,[inner_lane],color=[0,255,0])
    cv2.fillPoly(road_bkg,[left_lane],color=[255,255,255])
    cv2.fillPoly(road_bkg,[right_lane],color=[255,255,255])
    
    #Results screen portion for polynomial fit
    road1 = np.copy(road)
    cv2.putText(road1, 'Polynomial fit',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    
    road_warped = cv2.warpPerspective(road,Minv,img_size,flags=cv2.INTER_LINEAR)
    road_warped_bkg= cv2.warpPerspective(road_bkg,Minv,img_size,flags=cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(img,1.0,road_warped, -1.0, 0.0)
    result = cv2.addWeighted(base,1.0,road_warped, 1.0, 0.0)
    ym_per_pix = curve_centers.ym_per_pix # meters per pixel in y dimension
    xm_per_pix = curve_centers.xm_per_pix # meters per pixel in x dimension

    curve_fit_cr = np.polyfit(np.array(res_yvals,np.float32)*ym_per_pix,np.array(leftx,np.float32)*xm_per_pix,2)
    curverad = ((1 + (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])
    
    # Calculate the offset of the car on the road
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pos = 'right'

    # draw the text showing curvature, offset & speed
    cv2.putText(result, 'Radius of Curvature='+str(round(curverad,3))+'m ',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

    height, width = 1080, 1920
    FinalScreen = np.zeros((height, width, 3), dtype=np.uint8)
    FinalScreen[0:720,0:1280] = cv2.resize(result, (1280,720), interpolation=cv2.INTER_AREA)
    FinalScreen[0:360,1280:1920] = cv2.resize(warptrap, (640,360), interpolation=cv2.INTER_AREA)
    FinalScreen[360:720,1280:1920] = cv2.resize(binaryImage, (640,360), interpolation=cv2.INTER_AREA)
    FinalScreen[720:1080,1280:1920] = cv2.resize(warpage1, (640,360), interpolation=cv2.INTER_AREA)
    FinalScreen[720:1080,0:640] = cv2.resize(windowfit, (640,360), interpolation=cv2.INTER_AREA)
    FinalScreen[720:1080,640:1280] = cv2.resize(road1, (640,360), interpolation=cv2.INTER_AREA)
    return FinalScreen

Output_video = 'output1_tracked.mp4'
Input_video = 'project_video.mp4'
#Output_video = 'output_challenge_video.mp4'
#Input_video = 'harder_challenge_video.mp4'
#Output_video = 'output_challenge_video.mp4'
#Input_video = 'challenge_video.mp4'

clip1 = VideoFileClip(Input_video)
video_clip = clip1.fl_image(process_image) # This function expects color images
video_clip.write_videofile(Output_video, audio=False)


### Summary: The above solution works well on the standard video. However, it needs to be improved for the challenge videos. This is because the lanes are different in the challenge video: half of the lane is a freshly paved road and is different in color with the other other half of the lane. Also, the harder challenge video has glare on the camera as a result of direct sunlight falling on it, as well as high contrast contributing to washed out lane lines. This creates problems for the algorithm. One solution would be to dynamically adjust the contrast of the image frames dynamically to ensure images are not washed out and make sure they have a good dynamic range to work with in all lighting conditions. The harder challenge videos also have roads which are curvy and have a slope which makes it difficult to warp the images properly to feed into the algorithm. This can also be addressed by creating a dynamic region of interest for each image frame. These are some of the things that need to be explored when time permits. Overall, there is a lot of trail and error process in this project which makes it quite time consuming.